##1. Install and import needed stuff

In [ ]:
!pip install onnx
!pip install tf2onnx

In [ ]:
!wget https://chmura.put.poznan.pl/s/LxYSsota5PCbXcU/download
!unzip download

In [3]:
import os
import csv
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
import tf2onnx
import onnx
from random import shuffle
from google.colab import drive

## 2. Load Dataset

In [ ]:
drive.mount('/content/gdrive')

In [4]:
def load_dataset(directory):
    images = []
    labels = []

    for folder_name in os.listdir(directory):
        folder_path = os.path.join(directory, folder_name)
        if not os.path.isdir(folder_path):
            continue

        csv_path = os.path.join(directory, f"{folder_name}.csv")
        with open(csv_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)  # Skip header row
            for row in csv_reader:
                image_path = os.path.join(folder_path, f"{row[0]:0>4}.jpg")
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #image = cv2.resize(image, (56, 56))
                #image = np.transpose(image, (2, 0, 1))
                images.append(image)
                labels.append([float(row[1]), float(row[2])])

    return np.array(images), np.array(labels)

In [5]:
def preprocess_dataset(images, labels):
    images = np.array(images, dtype=np.float32) / 255.0
    labels = labels
    return images, labels

In [ ]:
dataset_directory = "/content/dataset"
images, labels = load_dataset(dataset_directory)
images, labels = preprocess_dataset(images, labels)

In [ ]:
ind_list = [i for i in range(len(images))]
shuffle(ind_list)
images  = images[ind_list, :,:,:]
labels = labels[ind_list, :]

In [ ]:
images = images[:1000]
labels = labels[:1000]

## 3. Data Augmentation

In [ ]:
import matplotlib as plt
import matplotlib.pyplot as plot
import random

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab.patches import cv2_imshow
  imshow = cv2_imshow
else:
  imshow = cv2.imshow

In [ ]:
def show_img(img):
    plot.axis("off")
    plot.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plot.show()

In [ ]:
def gamma(i, g):
    return i ** g

In [ ]:
images = [gamma(img, random.uniform(0.25, 1.75)) for img in images]

In [ ]:
show_img(images[0])

## 4. Train/Test/Val Split


In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42)

## 5. Create and Train Model

In [ ]:
model = Sequential()
model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='tanh'))

In [ ]:
model.compile(optimizer=Adam(), metrics = [tf.keras.metrics.Accuracy()], loss='mean_absolute_error')

In [ ]:
EPOCHS = 10
checkpoint_filepath = "/content/gdrive/MyDrive/roboticsII"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.load_weights(checkpoint_filepath)

In [ ]:
model.fit(train_images, train_labels, batch_size=32, epochs=EPOCHS, validation_data=(val_images, val_labels), callbacks = [model_checkpoint_callback])

Epoch 1/10
25/25 [==============================] - 16s 626ms/step - loss: 0.5773 - accuracy: 0.2956 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 2/10
25/25 [==============================] - 17s 677ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 3/10
25/25 [==============================] - 16s 633ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 4/10
25/25 [==============================] - 16s 627ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 5/10
25/25 [==============================] - 16s 632ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 6/10
25/25 [==============================] - 17s 667ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250
Epoch 7/10
25/25 [==============================] - 15s 617ms/step - loss: 0.5768 - accuracy: 0.3113 - val_loss: 0.5651 - val_accuracy: 0.3250

In [ ]:
#model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(val_images, val_labels))

In [ ]:
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=11, output_path="/content/model.onnx")

## 6. Make prediction

In [ ]:
prediction = model.predict(val_images)
print(prediction[:5])
print(val_labels[:5])

7/7 [==============================] - 2s 262ms/step
[[ 1. -1.]
 [ 1. -1.]
 [ 1. -1.]
 [ 1. -1.]
 [ 1. -1.]]
[[0.75      0.9921875]
 [0.6875    0.       ]
 [1.        0.       ]
 [1.        0.       ]
 [1.        0.       ]]


In [ ]:
print(prediction.max())
print(prediction.min())

1.0
-1.0


In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 222, 222, 8)       224       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 111, 111, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 109, 109, 16)      1168      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 54, 54, 16)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 46656)             0         
                                                                 
 dense_12 (Dense)            (None, 16)               